### A. Dataset

In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -P ./data/

In [2]:
import os 
import requests
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

In [3]:
# read the shakespear data
input_file_path = os.path.join(".", "data", 'input.txt')
with open(input_file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [4]:
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
print(f"train has {len(train_data):,} tokens")
print(f"val has {len(val_data):,} tokens")

train has 1,003,854 tokens
val has 111,540 tokens


In [7]:
# Context length is the input that transformer will see to make predictions
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target:{target}")

when input is tensor([18]) the target:47
when input is tensor([18, 47]) the target:56
when input is tensor([18, 47, 56]) the target:57
when input is tensor([18, 47, 56, 57]) the target:58
when input is tensor([18, 47, 56, 57, 58]) the target:1
when input is tensor([18, 47, 56, 57, 58,  1]) the target:15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target:47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target:58


In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Sample test
xb, yb = get_batch("train")
print(f"Inputs: {xb.shape}, \n {xb}")
print(f"Targets: {yb.shape},\n {yb}")

for b in range(batch_size):     # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"When input is {context.tolist()} the target: {target}")

Inputs: torch.Size([32, 8]), 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
   

### (B.) Model
- Bigram

In [9]:

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)

sample_idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(m.generate(idx=sample_idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.6922, device='cuda:0', grad_fn=<NllLossBackward0>)

jqfnxfRkRZ'Ndc.wf,ZWAO.zU,CbsK
bHiPWlkTBbzAuG:QaSKJO-33jMGF?KI3duM!bLVUYthgfjuDqca,xv.tbfF dXlAhcaAe


In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
for steps in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample batch of data 
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    #print(loss.item())

step 0: train loss 2.4536, val loss 2.4950
step 300: train loss 2.4594, val loss 2.4969
step 600: train loss 2.4512, val loss 2.4803
step 900: train loss 2.4512, val loss 2.4814
step 1200: train loss 2.4657, val loss 2.4879
step 1500: train loss 2.4551, val loss 2.4916
step 1800: train loss 2.4616, val loss 2.4840
step 2100: train loss 2.4577, val loss 2.4800
step 2400: train loss 2.4618, val loss 2.4813
step 2700: train loss 2.4611, val loss 2.4734


In [16]:
sample_idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(m.generate(idx=sample_idx, max_new_tokens=100)[0].tolist()))


BE:
Wileranousel lind pe l.
HAshe ce hiry:
Supr aisspllw y.
Herin's noroopetelaves
MP:

Pl, d mothak
